## Technical Indicators List

All the indicators that https://pypi.org/project/stockstats/ supports

In [1]:
INDICATORS_LIST_ALL = ['high_5_sma','rsi','boll','macd','cr','wr','cci','tr','atr','dma','pdi','dx','adx','adxr','trix','tema','vr','mfi','vwma',
                   'chop','ppo','stochrsi','supertrend','aroon','close_75_z','ao'
                   ]

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#%matplotlib inline
import mplfinance as mpf # matplot.finance to plot candlesticks
import random
import itertools # iterators for efficient looping: https://docs.python.org/3/library/itertools.html 

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader # a veces puede dar error al hacer fetch de la data porque no encuentra un stock o sus precios, o por la timezone
from finrl import config_tickers # config_tickers es una clase en la que se guardan en listas los nombres de cada stock para cada índice
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

/Users/carolinaalbamaruganrubio/miniforge3/envs/ai4finance/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TEST_START_DATE = '2021-10-01'
TEST_END_DATE = '2023-03-01'

In [4]:
stock = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Shape of DataFrame:  (97013, 8)


In [5]:
stock # this DF contains all the stock price history from the train start date to the test end date, for every firm in the DJ30

,date,open,high,low,close,volume,tic,day
0,2010-01-04,7.622500,7.660714,7.585000,6.470742,493729600,AAPL,0
1,2010-01-04,56.630001,57.869999,56.560001,41.493408,5277400,AMGN,0
2,2010-01-04,40.810001,41.099998,40.389999,33.090431,6894300,AXP,0
3,2010-01-04,55.720001,56.389999,54.799999,43.777546,6186700,BA,0
4,2010-01-04,57.650002,59.189999,57.509998,40.190224,7325600,CAT,0
...,...,...,...,...,...,...,...,...
97008,2023-02-28,482.670013,483.359985,473.920013,465.066833,3902100,UNH,1
97009,2023-02-28,220.000000,221.770004,219.500000,218.227646,5385400,V,1
97010,2023-02-28,38.700001,38.970001,38.549999,35.538387,16685300,VZ,1
97011,2023-02-28,35.480000,35.779999,35.320000,32.750977,8847000,WBA,1


In [6]:
stock.day.describe()

count    97013.000000
mean         2.024131
std          1.398530
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          4.000000
Name: day, dtype: float64

Stock symbols

In [7]:
tickers = stock.tic.unique()

We wanted to get:

In [8]:
print(np.reshape(config_tickers.DOW_30_TICKER, (len(config_tickers.DOW_30_TICKER))))
print("Number of firms in the index: ",len(config_tickers.DOW_30_TICKER))

['AXP' 'AMGN' 'AAPL' 'BA' 'CAT' 'CSCO' 'CVX' 'GS' 'HD' 'HON' 'IBM' 'INTC'
 'JNJ' 'KO' 'JPM' 'MCD' 'MMM' 'MRK' 'MSFT' 'NKE' 'PG' 'TRV' 'UNH' 'CRM'
 'VZ' 'V' 'WBA' 'WMT' 'DIS' 'DOW']
Number of firms in the index:  30


Creo que DOW es el DJIA, la media

But we got (because of the unavailable timezone or price data):

In [9]:
print(tickers) # stocks we have to train the agent
print("Number of firms downloaded: ",len(tickers))

['AAPL' 'AMGN' 'AXP' 'BA' 'CAT' 'CRM' 'CSCO' 'CVX' 'DIS' 'GS' 'HD' 'HON'
 'IBM' 'INTC' 'JNJ' 'JPM' 'KO' 'MCD' 'MMM' 'MRK' 'MSFT' 'NKE' 'PG' 'TRV'
 'UNH' 'V' 'VZ' 'WBA' 'WMT' 'DOW']
Number of firms downloaded:  30


## Agent 5

with following technical indicators:

In [32]:
# let's first take 1 technical indicators randomly from the supported indicators list
INDICATORS = [INDICATORS_LIST_ALL[i] for i in random.sample(range(len(INDICATORS_LIST_ALL)), 1)]

In [33]:
INDICATORS

['tr']

In [34]:
# creamos instancia de FeatureEngineer indicando la configuración deseada para hacer el preproceso
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True, # AUN NO SE QUE ES ESTO
                    use_turbulence=True, # measures extreme asset price fluctuation --> if the turbulence index reaches a pre-defined threshold, the agent will halt buying action and start selling the holding shares gradually
                    user_defined_feature = False) # MIRA A VER QUÉ ERA DEFINIRSE UNA FEATURE (creo que es )

processed = fe.preprocess_data(stock)

Successfully added technical indicators


[*********************100%%**********************]  1 of 1 completed


Shape of DataFrame:  (3310, 8)
Successfully added vix
Successfully added turbulence index


Comentario: hay indicadores que pese a estar documentados como soportados en https://pypi.org/project/stockstats/, esta versión de código no los reconoce. Habrá que comprobar mediante más pruebas que todos los incluídos en la lista de todos los indicadores, estén soportados por esta versión. 

In [35]:
processed

,date,open,high,low,close,volume,tic,day,tr,vix,turbulence
0,2010-01-04,7.622500,7.660714,7.585000,6.470742,493729600,AAPL,0,1.189972,20.040001,0.000000
1,2010-01-04,56.630001,57.869999,56.560001,41.493408,5277400,AMGN,0,16.376591,20.040001,0.000000
2,2010-01-04,40.810001,41.099998,40.389999,33.090431,6894300,AXP,0,8.009567,20.040001,0.000000
3,2010-01-04,55.720001,56.389999,54.799999,43.777546,6186700,BA,0,12.612453,20.040001,0.000000
4,2010-01-04,57.650002,59.189999,57.509998,40.190224,7325600,CAT,0,18.999775,20.040001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
95985,2023-02-27,488.769989,490.940002,481.959991,472.278198,3006200,UNH,0,17.674835,20.950001,7.163913
95986,2023-02-27,220.729996,221.440002,219.339996,218.634460,4255300,V,0,3.599304,20.950001,7.163913
95987,2023-02-27,38.990002,39.150002,38.630001,35.602486,14210900,VZ,0,3.675713,20.950001,7.163913
95988,2023-02-27,36.049999,36.080002,35.270000,32.621929,5580100,WBA,0,3.080139,20.950001,7.163913


In order to allow the Agent to have a consistent data structure to work with, we need to ensure that each stock has the same data range. This is necessary because not all stocks have data for every trading day due to holidays, stock-specific trading suspensions, or newly listed stocks. Therefore, we will establish a common data interval for each stock with day granularity and fill with 0 the (the more NaNs we have, the worst the performance will be). --> We can check this effect with other Stock Indices with less missing stock prices.

Convert the stock column to list

In [36]:
list_ticker = processed["tic"].unique().tolist()
print(list_ticker)
len(list_ticker)

['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT']


29

Create a list from the minimum to the maximum date and with the same granularity

In [37]:
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
list_date

['2010-01-04',
 '2010-01-05',
 '2010-01-06',
 '2010-01-07',
 '2010-01-08',
 '2010-01-09',
 '2010-01-10',
 '2010-01-11',
 '2010-01-12',
 '2010-01-13',
 '2010-01-14',
 '2010-01-15',
 '2010-01-16',
 '2010-01-17',
 '2010-01-18',
 '2010-01-19',
 '2010-01-20',
 '2010-01-21',
 '2010-01-22',
 '2010-01-23',
 '2010-01-24',
 '2010-01-25',
 '2010-01-26',
 '2010-01-27',
 '2010-01-28',
 '2010-01-29',
 '2010-01-30',
 '2010-01-31',
 '2010-02-01',
 '2010-02-02',
 '2010-02-03',
 '2010-02-04',
 '2010-02-05',
 '2010-02-06',
 '2010-02-07',
 '2010-02-08',
 '2010-02-09',
 '2010-02-10',
 '2010-02-11',
 '2010-02-12',
 '2010-02-13',
 '2010-02-14',
 '2010-02-15',
 '2010-02-16',
 '2010-02-17',
 '2010-02-18',
 '2010-02-19',
 '2010-02-20',
 '2010-02-21',
 '2010-02-22',
 '2010-02-23',
 '2010-02-24',
 '2010-02-25',
 '2010-02-26',
 '2010-02-27',
 '2010-02-28',
 '2010-03-01',
 '2010-03-02',
 '2010-03-03',
 '2010-03-04',
 '2010-03-05',
 '2010-03-06',
 '2010-03-07',
 '2010-03-08',
 '2010-03-09',
 '2010-03-10',
 '2010-03-

Assign every day to each stock

In [38]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2010-01-04', 'AAPL'),
 ('2010-01-04', 'AMGN'),
 ('2010-01-04', 'AXP'),
 ('2010-01-04', 'BA'),
 ('2010-01-04', 'CAT'),
 ('2010-01-04', 'CRM'),
 ('2010-01-04', 'CSCO'),
 ('2010-01-04', 'CVX'),
 ('2010-01-04', 'DIS'),
 ('2010-01-04', 'GS'),
 ('2010-01-04', 'HD'),
 ('2010-01-04', 'HON'),
 ('2010-01-04', 'IBM'),
 ('2010-01-04', 'INTC'),
 ('2010-01-04', 'JNJ'),
 ('2010-01-04', 'JPM'),
 ('2010-01-04', 'KO'),
 ('2010-01-04', 'MCD'),
 ('2010-01-04', 'MMM'),
 ('2010-01-04', 'MRK'),
 ('2010-01-04', 'MSFT'),
 ('2010-01-04', 'NKE'),
 ('2010-01-04', 'PG'),
 ('2010-01-04', 'TRV'),
 ('2010-01-04', 'UNH'),
 ('2010-01-04', 'V'),
 ('2010-01-04', 'VZ'),
 ('2010-01-04', 'WBA'),
 ('2010-01-04', 'WMT'),
 ('2010-01-05', 'AAPL'),
 ('2010-01-05', 'AMGN'),
 ('2010-01-05', 'AXP'),
 ('2010-01-05', 'BA'),
 ('2010-01-05', 'CAT'),
 ('2010-01-05', 'CRM'),
 ('2010-01-05', 'CSCO'),
 ('2010-01-05', 'CVX'),
 ('2010-01-05', 'DIS'),
 ('2010-01-05', 'GS'),
 ('2010-01-05', 'HD'),
 ('2010-01-05', 'HON'),
 ('2010-01-05', 'IB

Merge with stock DF (potentially resulting in NaN values)

In [39]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")

Remove non-trading days generated with the date range

In [40]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]

In [41]:
processed_full

,date,tic,open,high,low,close,volume,day,tr,vix,turbulence
0,2010-01-04,AAPL,7.622500,7.660714,7.585000,6.470742,493729600.0,0.0,1.189972,20.040001,0.000000
1,2010-01-04,AMGN,56.630001,57.869999,56.560001,41.493408,5277400.0,0.0,16.376591,20.040001,0.000000
2,2010-01-04,AXP,40.810001,41.099998,40.389999,33.090431,6894300.0,0.0,8.009567,20.040001,0.000000
3,2010-01-04,BA,55.720001,56.389999,54.799999,43.777546,6186700.0,0.0,12.612453,20.040001,0.000000
4,2010-01-04,CAT,57.650002,59.189999,57.509998,40.190224,7325600.0,0.0,18.999775,20.040001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
139282,2023-02-27,UNH,488.769989,490.940002,481.959991,472.278198,3006200.0,0.0,17.674835,20.950001,7.163913
139283,2023-02-27,V,220.729996,221.440002,219.339996,218.634460,4255300.0,0.0,3.599304,20.950001,7.163913
139284,2023-02-27,VZ,38.990002,39.150002,38.630001,35.602486,14210900.0,0.0,3.675713,20.950001,7.163913
139285,2023-02-27,WBA,36.049999,36.080002,35.270000,32.621929,5580100.0,0.0,3.080139,20.950001,7.163913


In [42]:
len(processed_full.tic.unique())

29

In [43]:
processed_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95990 entries, 0 to 139286
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        95990 non-null  object 
 1   tic         95990 non-null  object 
 2   open        95990 non-null  float64
 3   high        95990 non-null  float64
 4   low         95990 non-null  float64
 5   close       95990 non-null  float64
 6   volume      95990 non-null  float64
 7   day         95990 non-null  float64
 8   tr          95990 non-null  float64
 9   vix         95990 non-null  float64
 10  turbulence  95990 non-null  float64
dtypes: float64(9), object(2)
memory usage: 8.8+ MB


In [44]:
processed_full.describe()

,open,high,low,close,volume,day,tr,vix,turbulence
count,95990.000000,95990.000000,95990.000000,95990.000000,9.599000e+04,95990.000000,95990.000000,95990.000000,95990.000000
mean,102.224117,103.175365,101.253646,88.368877,1.995383e+07,2.024471,14.985354,18.721163,34.943607
std,73.520545,74.279832,72.738958,70.667737,6.204278e+07,1.398649,11.429568,7.269653,43.065959
min,6.870357,7.000000,6.794643,5.806766,3.054000e+05,0.000000,0.207500,9.140000,0.000000
25%,47.470001,47.910000,47.020000,38.991775,4.003909e+06,1.000000,6.292924,13.630000,14.973152
50%,83.498051,84.139999,82.830002,67.743038,7.250700e+06,2.000000,12.516567,16.875000,24.175786
75%,138.477531,139.731735,137.169998,118.837671,1.498672e+07,3.000000,20.462132,21.760000,40.000540
max,555.000000,558.099976,550.130005,540.800171,1.880998e+09,4.000000,79.644226,82.690002,652.614770


In this case, there are no nulls, but if there were we would need to fill them in the following way. 

In [45]:
processed_full = processed_full.sort_values(['date','tic']) # we need to sort bc of the data_split method CHECK SOURCE CODE 

processed_full = processed_full.fillna(0)

In [46]:
processed_full.describe()

,open,high,low,close,volume,day,tr,vix,turbulence
count,95990.000000,95990.000000,95990.000000,95990.000000,9.599000e+04,95990.000000,95990.000000,95990.000000,95990.000000
mean,102.224117,103.175365,101.253646,88.368877,1.995383e+07,2.024471,14.985354,18.721163,34.943607
std,73.520545,74.279832,72.738958,70.667737,6.204278e+07,1.398649,11.429568,7.269653,43.065959
min,6.870357,7.000000,6.794643,5.806766,3.054000e+05,0.000000,0.207500,9.140000,0.000000
25%,47.470001,47.910000,47.020000,38.991775,4.003909e+06,1.000000,6.292924,13.630000,14.973152
50%,83.498051,84.139999,82.830002,67.743038,7.250700e+06,2.000000,12.516567,16.875000,24.175786
75%,138.477531,139.731735,137.169998,118.837671,1.498672e+07,3.000000,20.462132,21.760000,40.000540
max,555.000000,558.099976,550.130005,540.800171,1.880998e+09,4.000000,79.644226,82.690002,652.614770


In [47]:
large_value = 1e9

processed_full.replace([np.inf], large_value, inplace=True)

In [48]:
processed_full.describe()

,open,high,low,close,volume,day,tr,vix,turbulence
count,95990.000000,95990.000000,95990.000000,95990.000000,9.599000e+04,95990.000000,95990.000000,95990.000000,95990.000000
mean,102.224117,103.175365,101.253646,88.368877,1.995383e+07,2.024471,14.985354,18.721163,34.943607
std,73.520545,74.279832,72.738958,70.667737,6.204278e+07,1.398649,11.429568,7.269653,43.065959
min,6.870357,7.000000,6.794643,5.806766,3.054000e+05,0.000000,0.207500,9.140000,0.000000
25%,47.470001,47.910000,47.020000,38.991775,4.003909e+06,1.000000,6.292924,13.630000,14.973152
50%,83.498051,84.139999,82.830002,67.743038,7.250700e+06,2.000000,12.516567,16.875000,24.175786
75%,138.477531,139.731735,137.169998,118.837671,1.498672e+07,3.000000,20.462132,21.760000,40.000540
max,555.000000,558.099976,550.130005,540.800171,1.880998e+09,4.000000,79.644226,82.690002,652.614770


Split Train and Test datasets

In [49]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
train

,date,tic,open,high,low,close,volume,day,tr,vix,turbulence
0,2010-01-04,AAPL,7.622500,7.660714,7.585000,6.470742,493729600.0,0.0,1.189972,20.040001,0.000000
0,2010-01-04,AMGN,56.630001,57.869999,56.560001,41.493408,5277400.0,0.0,16.376591,20.040001,0.000000
0,2010-01-04,AXP,40.810001,41.099998,40.389999,33.090431,6894300.0,0.0,8.009567,20.040001,0.000000
0,2010-01-04,BA,55.720001,56.389999,54.799999,43.777546,6186700.0,0.0,12.612453,20.040001,0.000000
0,2010-01-04,CAT,57.650002,59.189999,57.509998,40.190224,7325600.0,0.0,18.999775,20.040001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2956,2021-09-30,UNH,401.489990,403.489990,390.459991,375.759735,3779900.0,3.0,18.430969,23.139999,24.927454
2956,2021-09-30,V,227.580002,228.789993,222.630005,218.542923,7128500.0,3.0,6.391281,23.139999,24.927454
2956,2021-09-30,VZ,54.500000,54.509998,54.000000,45.622707,18736600.0,3.0,8.574741,23.139999,24.927454
2956,2021-09-30,WBA,48.790001,48.930000,46.919998,40.519112,6449400.0,3.0,6.981304,23.139999,24.927454


In [50]:
len(train.tic.unique())

29

In [51]:
train.tic.unique()

array(['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS',
       'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM',
       'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT'],
      dtype=object)

In [52]:
trade = data_split(processed_full, TEST_START_DATE,TEST_END_DATE)
trade

,date,tic,open,high,low,close,volume,day,tr,vix,turbulence
0,2021-10-01,AAPL,141.899994,142.919998,139.110001,140.653519,94639600.0,4.0,3.809998,21.100000,120.030307
0,2021-10-01,AMGN,213.589996,214.610001,210.800003,197.213058,2629400.0,4.0,18.567749,21.100000,120.030307
0,2021-10-01,AXP,168.500000,175.119995,168.479996,167.727478,3956000.0,4.0,13.573608,21.100000,120.030307
0,2021-10-01,BA,222.850006,226.720001,220.600006,226.000000,9113600.0,4.0,6.779999,21.100000,120.030307
0,2021-10-01,CAT,192.899994,195.869995,191.240005,183.514023,3695500.0,4.0,14.584641,21.100000,120.030307
...,...,...,...,...,...,...,...,...,...,...,...
352,2023-02-27,UNH,488.769989,490.940002,481.959991,472.278198,3006200.0,0.0,17.674835,20.950001,7.163913
352,2023-02-27,V,220.729996,221.440002,219.339996,218.634460,4255300.0,0.0,3.599304,20.950001,7.163913
352,2023-02-27,VZ,38.990002,39.150002,38.630001,35.602486,14210900.0,0.0,3.675713,20.950001,7.163913
352,2023-02-27,WBA,36.049999,36.080002,35.270000,32.621929,5580100.0,0.0,3.080139,20.950001,7.163913


In [53]:
train_length = len(train)
trade_length = len(trade)
print("Number of training samples: ", train_length)
print("Number of testing samples", trade_length)

Number of training samples:  85753
Number of testing samples 10237


In [54]:
train.tail()

,date,tic,open,high,low,close,volume,day,tr,vix,turbulence
2956,2021-09-30,UNH,401.489990,403.489990,390.459991,375.759735,3779900.0,3.0,18.430969,23.139999,24.927454
2956,2021-09-30,V,227.580002,228.789993,222.630005,218.542923,7128500.0,3.0,6.391281,23.139999,24.927454
2956,2021-09-30,VZ,54.500000,54.509998,54.000000,45.622707,18736600.0,3.0,8.574741,23.139999,24.927454
2956,2021-09-30,WBA,48.790001,48.930000,46.919998,40.519112,6449400.0,3.0,6.981304,23.139999,24.927454
2956,2021-09-30,WMT,46.880001,47.243332,46.416668,44.720795,22457700.0,3.0,2.182430,23.139999,24.927454


In [55]:
trade.head()

,date,tic,open,high,low,close,volume,day,tr,vix,turbulence
0,2021-10-01,AAPL,141.899994,142.919998,139.110001,140.653519,94639600.0,4.0,3.809998,21.1,120.030307
0,2021-10-01,AMGN,213.589996,214.610001,210.800003,197.213058,2629400.0,4.0,18.567749,21.1,120.030307
0,2021-10-01,AXP,168.500000,175.119995,168.479996,167.727478,3956000.0,4.0,13.573608,21.1,120.030307
0,2021-10-01,BA,222.850006,226.720001,220.600006,226.000000,9113600.0,4.0,6.779999,21.1,120.030307
0,2021-10-01,CAT,192.899994,195.869995,191.240005,183.514023,3695500.0,4.0,14.584641,21.1,120.030307


Since we need to set the parameters for the environment functions, we need to compute the stock dimension and state space.

In [56]:
# Number of unique stocks used for the training 
stock_dimension = len(train.tic.unique())
# {balance, close price, shares, N-technical indicators}
# Balance will occupy 1 input node, and it is computed as: balance = balance (t) − amount of money we pay to buy shares + amount of money we receive to sell shares
# We will have N input nodes for the stock prices and N additional input nodes to indicate the current number of shares for each stock --> 2*N
# we will have one node for every technical indicator for every stock --> M indicators * N stocks
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension (Number of different companies in which we want to invest initially): {stock_dimension}, State Space (Number of input nodes to feed to the network): {state_space}")

Stock Dimension (Number of different companies in which we want to invest initially): 29, State Space (Number of input nodes to feed to the network): 88


Most of the trading companies demand trading commissions or costs. Here, we will simulate a buying and selling commission of 0,1% for each transaction. We will assume the user starts having no shares of any stock. Let's also assume we have 1.000.000 dollars to invest with. We will also define a parameter that sets the maximum amount of shares to trade, h_max. It is interesting to set the scaling reward or gamma (importance we give to the future reward), so that we specify if we give more importance to the short or long term. We could modify these parameters, if needed.

In [57]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension
h_max = 100
initial_amount = 1000000
gamma = 1e-4 # we are assuming we want to get the highest return in the short term, preventing ourselves from the risk and uncertainty caused by external factors in the long term

HAZ PRUEBAS CON UN GAMMA MÁS ALTO

Let's set the configuration that will be passed to the Environment class

In [58]:
env_kwargs = {
    "hmax": h_max,
    "initial_amount": initial_amount, 
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": gamma
}

Let's create the Stock Trading Environment!

In [59]:
e_train_gym = StockTradingEnv(df = train, **env_kwargs) # creates instance of the Environment class
print("Environment class type", type(e_train_gym))
env_train, _ = e_train_gym.get_sb_env() # resets the environment and converts the initial environment into a DummyVecEnv instance
print("External Environment class type: ", type(env_train))

Environment class type <class 'finrl.meta.env_stock_trading.env_stocktrading.StockTradingEnv'>
External Environment class type:  <class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


Since we will use different RL algorithms, let's set some flags:

In [60]:
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = True
if_using_sac = True

Let's create the Deep Reinforcement Learning Agent!

podríamos runear todos los ind (p.e. 3 o 4 ind) y con los mejores, cambiar el resto de parametros (fechas, gamma,...)

# A2C

In [61]:
agent = DRLAgent(env = env_train) # creates Agent instance
model_a2c = agent.get_model("a2c") # gets stablebaselines3 model  

if if_using_a2c: # where to store the results
  # set up logger
  tmp_path = 'resultadosTFG' + '/set_5-' + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to resultadosTFG/set_5-/a2c


Let's train the agent!

In [62]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

---------------------------------------
| time/                 |             |
|    fps                | 156         |
|    iterations         | 100         |
|    time_elapsed       | 3           |
|    total_timesteps    | 500         |
| train/                |             |
|    entropy_loss       | -41.2       |
|    explained_variance | 0.0523      |
|    learning_rate      | 0.0007      |
|    n_updates          | 99          |
|    policy_loss        | -39.3       |
|    reward             | -0.30250955 |
|    std                | 1           |
|    value_loss         | 1.12        |
---------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 166       |
|    iterations         | 200       |
|    time_elapsed       | 5         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -41.2     |
|    explained_variance | -0.0929   |
|    learning_rate

In [63]:
trained_a2c.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=88, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=88, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=29, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

Let's see the learning progress of the agent

In [64]:
learning_process = pd.read_csv("resultadosTFG/set_5-/a2c/progress.csv")
learning_process

,time/fps,train/entropy_loss,time/time_elapsed,time/total_timesteps,train/n_updates,time/iterations,train/std,train/value_loss,train/reward,train/learning_rate,train/explained_variance,train/policy_loss
0,156,-41.209713,3,500,99,100,1.002259,1.117754,-0.302510,0.0007,5.225533e-02,-39.298534
1,166,-41.238201,5,1000,199,200,1.003399,3.384770,0.987190,0.0007,-9.285343e-02,-8.233973
2,172,-41.262112,8,1500,299,300,1.004422,3.258606,-1.932920,0.0007,-4.549651e-01,-24.611143
3,175,-41.261414,11,2000,399,400,1.004264,4.632020,-0.377233,0.0007,8.541059e-02,66.782143
4,177,-41.306908,14,2500,499,500,1.005743,5.061499,-1.250496,0.0007,0.000000e+00,25.001488
...,...,...,...,...,...,...,...,...,...,...,...,...
95,177,-43.066582,269,48000,9599,9600,1.071700,2.323151,-0.663229,0.0007,3.287893e-01,36.569008
96,177,-43.130432,272,48500,9699,9700,1.074083,1.685460,-1.851649,0.0007,0.000000e+00,54.055458
97,177,-43.169655,275,49000,9799,9800,1.075671,60.238079,-2.178310,0.0007,1.192093e-07,-311.728271
98,177,-43.241386,278,49500,9899,9900,1.078126,6.809357,-0.150231,0.0007,0.000000e+00,99.357674


Plot the training results

In [65]:
def plot_metrics(df, n_rows, n_cols):
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5 * n_cols, 5 * n_rows))
    axes = axes.flatten()  # Flatten the array of axes if more than one row or column

    # List of tuples containing (y-axis, title)
    metrics = [
        ('train/reward', 'Reward'),
        ('train/policy_loss', 'Policy Loss'),
        ('train/std', 'Standard Deviation'),
        ('train/n_updates', 'Number of Updates'),
        ('train/value_loss', 'Value Loss'),
        ('time/fps', 'Frames Per Second'),
        ('train/explained_variance', 'Explained Variance'),
        ('train/entropy_loss', 'Entropy Loss')
    ]

    # Plot each metric
    for ax, (metric, title) in zip(axes, metrics):
        ax.plot(df['time/total_timesteps'], df[metric], label=title)
        ax.set_xlabel('Total Timesteps')
        ax.set_ylabel(title)
        ax.set_title(title)
        ax.grid(True)
        ax.legend()

    plt.tight_layout()
    plt.savefig(f'resultadosTFG/set_5-/a2c/trainingStatsResults.png')
    plt.close()

df = learning_process
plot_metrics(df, n_rows=4, n_cols=2)  # Modify n_rows and n_cols as needed


The agent is not learning very well... 

Podríamos hacer una tabla resumen de todos los algos y pones cuántos steps han sido necesarios, la LR de cada uno, etc

# PPO

Set PPO parameters and set the logger

In [66]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = 'resultadosTFG' + '/set_5-' + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device
Logging to resultadosTFG/set_5-/ppo


Train PPO

In [67]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000) if if_using_ppo else None

----------------------------------
| time/              |           |
|    fps             | 200       |
|    iterations      | 1         |
|    time_elapsed    | 10        |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 0.8224327 |
----------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 205         |
|    iterations           | 2           |
|    time_elapsed         | 19          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014847258 |
|    clip_fraction        | 0.193       |
|    clip_range           | 0.2         |
|    entropy_loss         | -41.2       |
|    explained_variance   | 0.00894     |
|    learning_rate        | 0.00025     |
|    loss                 | 3.44        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0284     |
|    reward  

In [68]:
trained_ppo.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=88, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=88, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=29, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

Let's see the learning progress of the agent

In [69]:
learning_process = pd.read_csv("resultadosTFG/set_5-/ppo/progress.csv")
learning_process

,train/reward,time/fps,time/time_elapsed,time/total_timesteps,time/iterations,train/loss,train/policy_gradient_loss,train/std,train/clip_range,train/approx_kl,train/clip_fraction,train/value_loss,train/entropy_loss,train/n_updates,train/learning_rate,train/explained_variance
0,0.822433,200,10,2048,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.235802,205,19,4096,2,3.444308,-0.028430,1.001878,0.2,0.014847,0.192969,10.356352,-41.173859,10.0,0.00025,0.008944
2,2.073288,206,29,6144,3,33.696918,-0.016823,1.004120,0.2,0.015881,0.160840,44.232226,-41.230256,20.0,0.00025,0.018465
3,1.236013,206,39,8192,4,35.789394,-0.022202,1.005431,0.2,0.013927,0.143994,46.390814,-41.288633,30.0,0.00025,0.009061
4,-1.485536,206,49,10240,5,8.121819,-0.023071,1.008854,0.2,0.016068,0.183398,21.062403,-41.347582,40.0,0.00025,-0.055552
5,0.132184,205,59,12288,6,12.933814,-0.025768,1.011521,0.2,0.019121,0.225439,48.110785,-41.440150,50.0,0.00025,0.006851
6,2.257271,204,70,14336,7,25.013792,-0.019493,1.014638,0.2,0.015446,0.142334,95.277409,-41.520149,60.0,0.00025,0.005508
7,-1.866230,204,80,16384,8,9.899281,-0.022694,1.015758,0.2,0.016109,0.170068,17.280846,-41.587421,70.0,0.00025,0.005788
8,1.595853,204,90,18432,9,16.626333,-0.027905,1.019354,0.2,0.022595,0.243311,45.555854,-41.653288,80.0,0.00025,-0.009512
9,3.733602,204,100,20480,10,42.447430,-0.016081,1.021043,0.2,0.015683,0.161963,116.849226,-41.733772,90.0,0.00025,-0.010615


Plot the training results

In [70]:
def plot_metrics(df, n_rows, n_cols):
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5 * n_cols, 5 * n_rows))
    axes = axes.flatten()  # Flatten the array of axes if more than one row or column

    metrics = [
        ('train/reward', 'Reward'),
        ('train/loss', 'Loss'),
        ('train/std', 'Standard Deviation'),
        ('train/approx_kl', 'approximate Kullback-Leibler divergence'),
        ('train/clip_fraction', 'Clip Fraction'),
        ('train/clip_range', 'Clip Range'),
        ('train/explained_variance', 'Explained Variance'),
        ('train/value_loss', 'Value Loss'),
        ('train/policy_gradient_loss', 'Policy Gradient Loss'),
        ('train/entropy_loss', 'Entropy Loss')
    ]

    # Plot each metric
    for ax, (metric, title) in zip(axes, metrics):
        ax.plot(df['time/total_timesteps'], df[metric], label=title)
        ax.set_xlabel('Total Timesteps')
        ax.set_ylabel(title)
        ax.set_title(title)
        ax.grid(True)
        ax.legend()

    plt.tight_layout()
    plt.savefig(f'resultadosTFG/set_5-/ppo/trainingStatsResults.png')
    plt.close()

df = learning_process
plot_metrics(df, n_rows=5, n_cols=2)  # Modify n_rows and n_cols as needed

# DDPG

Create the agent and set the logger

In [71]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
  # set up logger
  tmp_path = 'resultadosTFG' + '/set_5-' + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device
Logging to resultadosTFG/set_5-/ddpg


Train the agent

In [72]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000) if if_using_ddpg else None

day: 2956, episode: 40
begin_total_asset: 1000000.00
end_total_asset: 5012869.06
total_reward: 4012869.06
total_cost: 998.99
total_trades: 41384
Sharpe: 0.874
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 103       |
|    time_elapsed    | 114       |
|    total_timesteps | 11828     |
| train/             |           |
|    actor_loss      | 22.6      |
|    critic_loss     | 4.59e+03  |
|    learning_rate   | 0.001     |
|    n_updates       | 8871      |
|    reward          | -8.237783 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 94        |
|    time_elapsed    | 250       |
|    total_timesteps | 23656     |
| train/             |           |
|    actor_loss      | -5.61     |
|    critic_loss     | 77.9      |
|    learning_rate   | 0.001     |
|    n_updates       | 20699     |
|

In [73]:
trained_ddpg.policy

TD3Policy(
  (actor): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (mu): Sequential(
      (0): Linear(in_features=88, out_features=400, bias=True)
      (1): ReLU()
      (2): Linear(in_features=400, out_features=300, bias=True)
      (3): ReLU()
      (4): Linear(in_features=300, out_features=29, bias=True)
      (5): Tanh()
    )
  )
  (actor_target): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (mu): Sequential(
      (0): Linear(in_features=88, out_features=400, bias=True)
      (1): ReLU()
      (2): Linear(in_features=400, out_features=300, bias=True)
      (3): ReLU()
      (4): Linear(in_features=300, out_features=29, bias=True)
      (5): Tanh()
    )
  )
  (critic): ContinuousCritic(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (qf0): Sequential(
      (0): Linear(in_features=117, out_fe

Let's see the learning progress of the agent 

In [74]:
learning_process = pd.read_csv("resultadosTFG/set_5-/ddpg/progress.csv")
learning_process

,train/critic_loss,time/fps,time/time_elapsed,time/total_timesteps,train/n_updates,train/reward,train/actor_loss,train/learning_rate,time/episodes
0,4586.125712,103,114,11828,8871,-8.237783,22.607354,0.001,4
1,77.852097,94,250,23656,20699,-8.237783,-5.606941,0.001,8
2,9.033052,92,382,35484,32527,-8.237783,-12.648343,0.001,12
3,6.508533,91,516,47312,44355,-8.237783,-13.986014,0.001,16


Plot the results of the training

In [75]:
def plot_metrics(df, n_rows, n_cols):
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5 * n_cols, 5 * n_rows))
    axes = axes.flatten()  # Flatten the array of axes if more than one row or column

    metrics = [
        ('train/reward', 'Reward'),
        ('train/actor_loss', 'Actor Loss'),
        ('train/n_updates', 'Number of Updates'),
        ('train/critic_loss', 'Critic Loss')
    ]

    # Plot each metric
    for ax, (metric, title) in zip(axes, metrics):
        ax.plot(df['time/total_timesteps'], df[metric], label=title)
        ax.set_xlabel('Total Timesteps')
        ax.set_ylabel(title)
        ax.set_title(title)
        ax.grid(True)
        ax.legend()

    plt.tight_layout()
    plt.savefig(f'resultadosTFG/set_5-/ddpg/trainingStatsResults.png')
    plt.close()

df = learning_process
plot_metrics(df, n_rows=4, n_cols=1)  # Modify n_rows and n_cols as needed

# TD3

Create the agent and set the logger

In [76]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = 'resultadosTFG' + '/set_5-' + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device
Logging to resultadosTFG/set_5-/td3


Train TD3

In [77]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000) if if_using_td3 else None

-----------------------------------
| time/              |            |
|    episodes        | 4          |
|    fps             | 101        |
|    time_elapsed    | 116        |
|    total_timesteps | 11828      |
| train/             |            |
|    actor_loss      | 90.9       |
|    critic_loss     | 3.01e+03   |
|    learning_rate   | 0.001      |
|    n_updates       | 8871       |
|    reward          | -6.6792808 |
-----------------------------------
day: 2956, episode: 60
begin_total_asset: 1000000.00
end_total_asset: 4339647.12
total_reward: 3339647.12
total_cost: 1163.71
total_trades: 50291
Sharpe: 0.782
-----------------------------------
| time/              |            |
|    episodes        | 8          |
|    fps             | 94         |
|    time_elapsed    | 250        |
|    total_timesteps | 23656      |
| train/             |            |
|    actor_loss      | 120        |
|    critic_loss     | 224        |
|    learning_rate   | 0.001      |
|    n_updat

In [78]:
trained_td3.policy

TD3Policy(
  (actor): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (mu): Sequential(
      (0): Linear(in_features=88, out_features=400, bias=True)
      (1): ReLU()
      (2): Linear(in_features=400, out_features=300, bias=True)
      (3): ReLU()
      (4): Linear(in_features=300, out_features=29, bias=True)
      (5): Tanh()
    )
  )
  (actor_target): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (mu): Sequential(
      (0): Linear(in_features=88, out_features=400, bias=True)
      (1): ReLU()
      (2): Linear(in_features=400, out_features=300, bias=True)
      (3): ReLU()
      (4): Linear(in_features=300, out_features=29, bias=True)
      (5): Tanh()
    )
  )
  (critic): ContinuousCritic(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (qf0): Sequential(
      (0): Linear(in_features=117, out_fe

Let's see agent's learning process

In [79]:
learning_process = pd.read_csv("resultadosTFG/set_5-/td3/progress.csv")
learning_process

,train/critic_loss,time/fps,time/time_elapsed,time/total_timesteps,train/n_updates,train/reward,train/actor_loss,train/learning_rate,time/episodes
0,3008.496143,101,116,11828,8871,-6.679281,90.936481,0.001,4
1,223.890119,94,250,23656,20699,-6.679281,120.305649,0.001,8
2,24.104260,90,392,35484,32527,-6.679281,91.465417,0.001,12
3,15.397743,89,526,47312,44355,-6.679281,64.638529,0.001,16


Plot training results

In [80]:
def plot_metrics(df, n_rows, n_cols):
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5 * n_cols, 5 * n_rows))
    axes = axes.flatten()  # Flatten the array of axes if more than one row or column

    metrics = [
        ('train/reward', 'Reward'),
        ('train/actor_loss', 'Actor Loss'),
        ('train/n_updates', 'Number of Updates'),
        ('train/critic_loss', 'Critic Loss')
    ]

    # Plot each metric
    for ax, (metric, title) in zip(axes, metrics):
        ax.plot(df['time/total_timesteps'], df[metric], label=title)
        ax.set_xlabel('Total Timesteps')
        ax.set_ylabel(title)
        ax.set_title(title)
        ax.grid(True)
        ax.legend()

    plt.tight_layout()
    plt.savefig(f'resultadosTFG/set_5-/td3/trainingStatsResults.png')
    plt.close()

df = learning_process
plot_metrics(df, n_rows=4, n_cols=1)  # Modify n_rows and n_cols as needed

# SAC

Create the agent, set parameters for training, and set logger

In [81]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = 'resultadosTFG' + '/set_5-' + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device
Logging to resultadosTFG/set_5-/sac


Train SAC

In [82]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=50000) if if_using_sac else None

----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 63        |
|    time_elapsed    | 185       |
|    total_timesteps | 11828     |
| train/             |           |
|    actor_loss      | 2.01e+03  |
|    critic_loss     | 501       |
|    ent_coef        | 0.291     |
|    ent_coef_loss   | 117       |
|    learning_rate   | 0.0001    |
|    n_updates       | 11727     |
|    reward          | -8.658614 |
----------------------------------
day: 2956, episode: 80
begin_total_asset: 1000000.00
end_total_asset: 5905277.89
total_reward: 4905277.89
total_cost: 14112.40
total_trades: 50745
Sharpe: 0.922
-----------------------------------
| time/              |            |
|    episodes        | 8          |
|    fps             | 62         |
|    time_elapsed    | 379        |
|    total_timesteps | 23656      |
| train/             |            |
|    actor_loss      | 860        |
|    critic_loss     | 86.

In [83]:
trained_sac.policy

SACPolicy(
  (actor): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (latent_pi): Sequential(
      (0): Linear(in_features=88, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
    )
    (mu): Linear(in_features=256, out_features=29, bias=True)
    (log_std): Linear(in_features=256, out_features=29, bias=True)
  )
  (critic): ContinuousCritic(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (qf0): Sequential(
      (0): Linear(in_features=117, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=1, bias=True)
    )
    (qf1): Sequential(
      (0): Linear(in_features=117, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=25

Let's see agent's learning process

In [84]:
learning_process = pd.read_csv("resultadosTFG/set_5-/sac/progress.csv")
learning_process

,train/critic_loss,time/fps,train/ent_coef,time/time_elapsed,time/total_timesteps,train/n_updates,time/episodes,train/reward,train/actor_loss,train/learning_rate,train/ent_coef_loss
0,501.487671,63,0.290824,185,11828,11727,4,-8.658614,2006.761353,0.0001,117.205856
1,86.591339,62,0.104272,379,23656,23555,8,-9.453074,859.612976,0.0001,-81.615479
2,63.758842,60,0.033269,584,35484,35383,12,-7.984707,364.579346,0.0001,-83.784653
3,361.247009,59,0.010845,791,47312,47211,16,-7.303645,187.718063,0.0001,-60.781395


Agent's training results

In [85]:
def plot_metrics(df, n_rows, n_cols):
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5 * n_cols, 5 * n_rows))
    axes = axes.flatten()  # Flatten the array of axes if more than one row or column

    metrics = [
        ('train/reward', 'Reward'),
        ('train/actor_loss', 'Actor Loss'),
        ('train/ent_coef_loss', 'Entropy coefficient Loss'),
        ('train/ent_coef', 'Entropy coefficient'),
        ('train/critic_loss', 'Critic Loss')
    ]

    # Plot each metric
    for ax, (metric, title) in zip(axes, metrics):
        ax.plot(df['time/total_timesteps'], df[metric], label=title)
        ax.set_xlabel('Total Timesteps')
        ax.set_ylabel(title)
        ax.set_title(title)
        ax.grid(True)
        ax.legend()

    plt.tight_layout()
    plt.savefig(f'resultadosTFG/set_5-/sac/trainingStatsResults.png')
    plt.close()

df = learning_process
plot_metrics(df, n_rows=5, n_cols=1)  # Modify n_rows and n_cols as needed

# Testing

There is a new environment for testing

We feed the environment with the testing dataset we prepared before (from start test date to end test date: with closing prices, etc)

In [86]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
e_trade_gym

Environment from Stable baselines3; observations is a matrix containing the inputs {balance, close price, shares, N-technical indicators}

In [87]:
env_trade, obs_trade = e_trade_gym.get_sb_env()
env_trade

In [88]:
obs_trade

array([[1.00000000e+06, 1.40653519e+02, 1.97213058e+02, 1.67727478e+02,
        2.26000000e+02, 1.83514023e+02, 2.74900909e+02, 5.07116737e+01,
        9.47543640e+01, 1.75438217e+02, 3.52330719e+02, 3.07770996e+02,
        2.04700592e+02, 1.21619804e+02, 5.01258240e+01, 1.48433228e+02,
        1.54524490e+02, 4.87658348e+01, 2.28277756e+02, 1.26340286e+02,
        7.50690918e+01, 2.82857422e+02, 1.42315781e+02, 1.30338409e+02,
        1.43675735e+02, 3.77384949e+02, 2.26107330e+02, 4.58676834e+01,
        4.03038101e+01, 4.39732056e+01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

In [89]:
trade

,date,tic,open,high,low,close,volume,day,tr,vix,turbulence
0,2021-10-01,AAPL,141.899994,142.919998,139.110001,140.653519,94639600.0,4.0,3.809998,21.100000,120.030307
0,2021-10-01,AMGN,213.589996,214.610001,210.800003,197.213058,2629400.0,4.0,18.567749,21.100000,120.030307
0,2021-10-01,AXP,168.500000,175.119995,168.479996,167.727478,3956000.0,4.0,13.573608,21.100000,120.030307
0,2021-10-01,BA,222.850006,226.720001,220.600006,226.000000,9113600.0,4.0,6.779999,21.100000,120.030307
0,2021-10-01,CAT,192.899994,195.869995,191.240005,183.514023,3695500.0,4.0,14.584641,21.100000,120.030307
...,...,...,...,...,...,...,...,...,...,...,...
352,2023-02-27,UNH,488.769989,490.940002,481.959991,472.278198,3006200.0,0.0,17.674835,20.950001,7.163913
352,2023-02-27,V,220.729996,221.440002,219.339996,218.634460,4255300.0,0.0,3.599304,20.950001,7.163913
352,2023-02-27,VZ,38.990002,39.150002,38.630001,35.602486,14210900.0,0.0,3.675713,20.950001,7.163913
352,2023-02-27,WBA,36.049999,36.080002,35.270000,32.621929,5580100.0,0.0,3.080139,20.950001,7.163913


## A2C

Test, predict

In [90]:
trained_model = trained_a2c
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_model, 
    environment = e_trade_gym)

el modelo es 
estos son los test obs
[[1.00000000e+06 1.40653519e+02 1.97213058e+02 1.67727478e+02
  2.26000000e+02 1.83514023e+02 2.74900909e+02 5.07116737e+01
  9.47543640e+01 1.75438217e+02 3.52330719e+02 3.07770996e+02
  2.04700592e+02 1.21619804e+02 5.01258240e+01 1.48433228e+02
  1.54524490e+02 4.87658348e+01 2.28277756e+02 1.26340286e+02
  7.50690918e+01 2.82857422e+02 1.42315781e+02 1.30338409e+02
  1.43675735e+02 3.77384949e+02 2.26107330e+02 4.58676834e+01
  4.03038101e+01 4.39732056e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 3.80999756e+00
  1.85677490e+01 1.35736084e+01 6

Plot and Save testing results, Portfolio Value

In [91]:
df_account_value_a2c['date'] = pd.to_datetime(df_account_value_a2c['date']).dt.normalize()
plt.figure(figsize=(14, 7))
plt.plot(df_account_value_a2c['date'], df_account_value_a2c['account_value'])

# Setting major locator
locator = mdates.MonthLocator(interval=1)  # Show a tick every week
plt.gca().xaxis.set_major_locator(locator)

# Setting formatter
formatter = mdates.DateFormatter('%Y-%m-%d')
plt.gca().xaxis.set_major_formatter(formatter)

# Optionally, auto-format date labels to prevent overlap
plt.gcf().autofmt_xdate()

plt.title('Account Value Testing for A2C')
plt.xlabel('Date')
plt.ylabel('Account Value')
plt.grid(True)

plt.savefig(f'resultadosTFG/set_5-/a2c/testing/accountValue.png')
plt.close()

Trades made in testing

In [92]:
display(df_actions_a2c)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2021-10-01,0,100,0,0,0,4,100,0,0,100,...,0,0,100,0,55,0,7,100,100,0
2021-10-04,0,100,0,0,0,4,100,0,0,100,...,0,0,100,0,55,0,7,100,100,0
2021-10-05,0,100,0,0,0,4,100,0,0,100,...,0,0,100,0,55,0,7,100,100,0
2021-10-06,0,100,0,0,0,4,100,0,0,100,...,0,0,100,0,55,0,7,100,100,0
2021-10-07,0,100,0,0,0,4,100,0,0,100,...,0,0,100,0,55,0,7,100,100,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## PPO

Test, predict

In [93]:
trained_model = trained_ppo
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_model, 
    environment = e_trade_gym)

el modelo es 
estos son los test obs
[[1.00000000e+06 1.40653519e+02 1.97213058e+02 1.67727478e+02
  2.26000000e+02 1.83514023e+02 2.74900909e+02 5.07116737e+01
  9.47543640e+01 1.75438217e+02 3.52330719e+02 3.07770996e+02
  2.04700592e+02 1.21619804e+02 5.01258240e+01 1.48433228e+02
  1.54524490e+02 4.87658348e+01 2.28277756e+02 1.26340286e+02
  7.50690918e+01 2.82857422e+02 1.42315781e+02 1.30338409e+02
  1.43675735e+02 3.77384949e+02 2.26107330e+02 4.58676834e+01
  4.03038101e+01 4.39732056e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 3.80999756e+00
  1.85677490e+01 1.35736084e+01 6

Save Plot of Portfolio Value

In [94]:
df_account_value_ppo['date'] = pd.to_datetime(df_account_value_ppo['date']).dt.normalize()
plt.figure(figsize=(14, 7))
plt.plot(df_account_value_ppo['date'], df_account_value_ppo['account_value'])

# Setting major locator
locator = mdates.MonthLocator(interval=1)  # Show a tick every week
plt.gca().xaxis.set_major_locator(locator)

# Setting formatter
formatter = mdates.DateFormatter('%Y-%m-%d')
plt.gca().xaxis.set_major_formatter(formatter)

# Optionally, auto-format date labels to prevent overlap
plt.gcf().autofmt_xdate()

plt.title('Account Value Testing for PPO')
plt.xlabel('Date')
plt.ylabel('Account Value')
plt.grid(True)

plt.savefig(f'resultadosTFG/set_5-/ppo/testing/accountValue.png')
plt.close()


Trades made in testing

In [95]:
df_actions_ppo.tail(40)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2022-12-28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-12-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-12-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## DDPG

Test DDPG, predict

In [96]:
trained_model = trained_ddpg
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_model, 
    environment = e_trade_gym)

el modelo es 
estos son los test obs
[[1.00000000e+06 1.40653519e+02 1.97213058e+02 1.67727478e+02
  2.26000000e+02 1.83514023e+02 2.74900909e+02 5.07116737e+01
  9.47543640e+01 1.75438217e+02 3.52330719e+02 3.07770996e+02
  2.04700592e+02 1.21619804e+02 5.01258240e+01 1.48433228e+02
  1.54524490e+02 4.87658348e+01 2.28277756e+02 1.26340286e+02
  7.50690918e+01 2.82857422e+02 1.42315781e+02 1.30338409e+02
  1.43675735e+02 3.77384949e+02 2.26107330e+02 4.58676834e+01
  4.03038101e+01 4.39732056e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 3.80999756e+00
  1.85677490e+01 1.35736084e+01 6

Store graph of the value of portfolio

In [97]:
df_account_value_ddpg['date'] = pd.to_datetime(df_account_value_ddpg['date']).dt.normalize()
plt.figure(figsize=(14, 7))
plt.plot(df_account_value_ddpg['date'], df_account_value_ddpg['account_value'])

# Setting major locator
locator = mdates.MonthLocator(interval=1)  # Show a tick every week
plt.gca().xaxis.set_major_locator(locator)

# Setting formatter
formatter = mdates.DateFormatter('%Y-%m-%d')
plt.gca().xaxis.set_major_formatter(formatter)

# Optionally, auto-format date labels to prevent overlap
plt.gcf().autofmt_xdate()

plt.title('Account Value Testing for DDPG')
plt.xlabel('Date')
plt.ylabel('Account Value')
plt.grid(True)

plt.savefig(f'resultadosTFG/set_5-/ddpg/testing/accountValue.png')
plt.close()


Trades made in testing

In [98]:
df_actions_ddpg.head(30)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2021-10-01,0,100,100,0,0,0,0,0,0,100,...,100,100,100,100,100,100,100,100,0,0
2021-10-04,0,100,100,0,0,0,0,0,0,100,...,100,100,100,100,100,100,100,100,0,0
2021-10-05,0,100,100,0,0,0,0,0,0,100,...,100,100,100,100,100,100,100,100,0,0
2021-10-06,0,100,100,0,0,0,0,0,0,21,...,100,100,100,100,100,100,100,100,0,0
2021-10-07,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2021-10-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-10-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-10-12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-10-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## TD3

Test TD3, predict

In [99]:
trained_model = trained_td3
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_model, 
    environment = e_trade_gym)

el modelo es 
estos son los test obs
[[1.00000000e+06 1.40653519e+02 1.97213058e+02 1.67727478e+02
  2.26000000e+02 1.83514023e+02 2.74900909e+02 5.07116737e+01
  9.47543640e+01 1.75438217e+02 3.52330719e+02 3.07770996e+02
  2.04700592e+02 1.21619804e+02 5.01258240e+01 1.48433228e+02
  1.54524490e+02 4.87658348e+01 2.28277756e+02 1.26340286e+02
  7.50690918e+01 2.82857422e+02 1.42315781e+02 1.30338409e+02
  1.43675735e+02 3.77384949e+02 2.26107330e+02 4.58676834e+01
  4.03038101e+01 4.39732056e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 3.80999756e+00
  1.85677490e+01 1.35736084e+01 6

Store graph of the value of portfolio

In [100]:
df_account_value_td3['date'] = pd.to_datetime(df_account_value_td3['date']).dt.normalize()
plt.figure(figsize=(14, 7))
plt.plot(df_account_value_td3['date'], df_account_value_td3['account_value'])

# Setting major locator
locator = mdates.MonthLocator(interval=1)  # Show a tick every week
plt.gca().xaxis.set_major_locator(locator)

# Setting formatter
formatter = mdates.DateFormatter('%Y-%m-%d')
plt.gca().xaxis.set_major_formatter(formatter)

# Optionally, auto-format date labels to prevent overlap
plt.gcf().autofmt_xdate()

plt.title('Account Value Testing for TD3')
plt.xlabel('Date')
plt.ylabel('Account Value')
plt.grid(True)

plt.savefig(f'resultadosTFG/set_5-/td3/testing/accountValue.png')
plt.close()


Trades made in testing

In [101]:
df_actions_td3.head(30)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2021-10-01,0,0,0,100,0,100,0,100,100,100,...,0,0,100,100,0,100,100,100,100,100
2021-10-04,0,0,0,100,0,100,0,100,100,100,...,0,0,100,100,0,100,100,100,100,100
2021-10-05,0,0,0,100,0,100,0,100,100,100,...,0,0,100,100,0,100,100,100,100,100
2021-10-06,0,0,0,0,0,96,0,0,100,100,...,0,0,0,0,0,0,0,0,0,100
2021-10-07,0,0,0,100,0,0,0,0,0,0,...,0,0,-99,0,0,0,0,0,2,100
2021-10-08,0,0,0,100,0,0,0,0,0,0,...,0,0,-99,0,0,0,0,0,2,100
2021-10-11,0,0,0,99,0,0,0,0,0,0,...,0,0,-99,0,0,0,0,0,1,100
2021-10-12,0,0,0,0,0,1,0,0,0,9,...,0,0,-3,0,0,0,0,0,1,100
2021-10-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## SAC

Test SAC, predict

Account value is the DF that stores the history of the value of the assets (portfolio value)

In [102]:
trained_model = trained_sac
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_model, 
    environment = e_trade_gym)

el modelo es 
estos son los test obs
[[1.00000000e+06 1.40653519e+02 1.97213058e+02 1.67727478e+02
  2.26000000e+02 1.83514023e+02 2.74900909e+02 5.07116737e+01
  9.47543640e+01 1.75438217e+02 3.52330719e+02 3.07770996e+02
  2.04700592e+02 1.21619804e+02 5.01258240e+01 1.48433228e+02
  1.54524490e+02 4.87658348e+01 2.28277756e+02 1.26340286e+02
  7.50690918e+01 2.82857422e+02 1.42315781e+02 1.30338409e+02
  1.43675735e+02 3.77384949e+02 2.26107330e+02 4.58676834e+01
  4.03038101e+01 4.39732056e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 3.80999756e+00
  1.85677490e+01 1.35736084e+01 6

Store graph of the value of portfolio

In [103]:
df_account_value_sac['date'] = pd.to_datetime(df_account_value_sac['date']).dt.normalize()
plt.figure(figsize=(14, 7))
plt.plot(df_account_value_sac['date'], df_account_value_sac['account_value'])

# Setting major locator
locator = mdates.MonthLocator(interval=1)  # Show a tick every week
plt.gca().xaxis.set_major_locator(locator)

# Setting formatter
formatter = mdates.DateFormatter('%Y-%m-%d')
plt.gca().xaxis.set_major_formatter(formatter)

# Optionally, auto-format date labels to prevent overlap
plt.gcf().autofmt_xdate()

plt.title('Account Value Testing for SAC')
plt.xlabel('Date')
plt.ylabel('Account Value')
plt.grid(True)

plt.savefig(f'resultadosTFG/set_5-/sac/testing/accountValue.png')
plt.close()


Trades made in testing

In [104]:
df_actions_sac.head(30)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2021-10-01,82,0,0,0,0,0,0,96,0,0,...,92,56,0,0,0,48,96,0,95,0
2021-10-04,82,0,0,0,0,0,0,96,0,0,...,92,56,0,0,0,48,96,0,95,0
2021-10-05,82,0,0,0,0,0,0,96,0,0,...,92,56,0,0,0,48,96,0,95,0
2021-10-06,82,0,0,0,0,0,0,96,0,0,...,92,56,0,0,0,48,96,0,95,0
2021-10-07,82,0,0,0,0,0,0,96,0,0,...,92,56,0,0,0,48,96,0,95,0
2021-10-08,82,0,0,0,0,0,0,96,0,0,...,92,0,0,0,0,0,96,0,95,0
2021-10-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-10-12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-10-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Obtain results to then merge: merge in one DF the Portfolio Value given by each algorithm

In [105]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
df_result_a2c.rename(columns = {'account_value':'a2c'}, inplace = True)
df_result_ddpg = df_account_value_ddpg.set_index(df_account_value_ddpg.columns[0])
df_result_ddpg.rename(columns = {'account_value':'ddpg'}, inplace = True)
df_result_td3 = df_account_value_td3.set_index(df_account_value_td3.columns[0])
df_result_td3.rename(columns = {'account_value':'td3'}, inplace = True)
df_result_ppo = df_account_value_ppo.set_index(df_account_value_ppo.columns[0])
df_result_ppo.rename(columns = {'account_value':'ppo'}, inplace = True)
df_result_sac = df_account_value_sac.set_index(df_account_value_sac.columns[0])
df_result_sac.rename(columns = {'account_value':'sac'}, inplace = True)

We can also get stats from each stock individually (even though the analysis won't focus on that)

In [106]:
#baseline stats

print("==============Get Baseline Stats===========")
print("==============Baseline Stats of Profitability from DJIA===========")
print("This is the real performance of the index")
df_dji_ = get_baseline(
        ticker="^DJI", 
        start = TEST_START_DATE,
        end = TEST_END_DATE)
stats = backtest_stats(df_dji_, value_col_name = 'close')

==============Get Baseline Stats===========
==============Baseline Stats of Profitability from DJIA===========
This is the real performance of the index


[*********************100%%**********************]  1 of 1 completed

Shape of DataFrame:  (354, 8)
Annual return         -0.034876
Cumulative returns    -0.048644
Annual volatility      0.181612
Sharpe ratio          -0.105351
Calmar ratio          -0.158953
Stability              0.280983
Max drawdown          -0.219408
Omega ratio            0.982546
Sortino ratio         -0.146974
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.970602
Daily value at risk   -0.022957
dtype: float64


get_baseline vuelve a fetchear los stocks de YahooFinance dadas esas fechas.

backtest_stats llama a get daily return y luego saca stats de esa Serie con metodo que pide:

    """
    Calculates various performance metrics of a strategy, for use in
    plotting.show_perf_stats.

    Parameters
    ----------
    returns : pd.Series
        Daily returns of the strategy, noncumulative.
         - See full explanation in tears.create_full_tear_sheet.
    factor_returns : pd.Series, optional
        Daily noncumulative returns of the benchmark factor to which betas are
        computed. Usually a benchmark such as market returns.
         - This is in the same style as returns.
         - If None, do not compute alpha, beta, and information ratio.
    positions : pd.DataFrame
        Daily net position values.
         - See full explanation in tears.create_full_tear_sheet.
    transactions : pd.DataFrame
        Prices and amounts of executed trades. One row per trade.
        - See full explanation in tears.create_full_tear_sheet.
    turnover_denom : str
        Either AGB or portfolio_value, default AGB.
        - See full explanation in txn.get_turnover.

    Returns
    -------
    pd.Series
        Performance metrics.
    """

get daily profitability * invested amount = portfolio value in case of investing 1000000$ on DJ and not touching it

In [107]:
df_dji = pd.DataFrame()
df_dji['date'] = df_account_value_a2c['date']
df_dji['account_value'] = df_dji_['close'] / df_dji_['close'][0] * env_kwargs["initial_amount"] # get daily profitability * invested amount = portfolio value in case of investing 1000000$ on DJ and not touching it
#df_dji.to_csv("df_dji.csv")
df_dji = df_dji.set_index(df_dji.columns[0])
#df_dji.to_csv("df_dji+.csv")

In [108]:
df_dji.rename(columns = {'account_value':'dji'}, inplace = True)

In [109]:
df_result_td3

,td3
date,
2021-10-01,1.000000e+06
2021-10-04,9.969719e+05
2021-10-05,1.002713e+06
2021-10-06,1.006578e+06
2021-10-07,1.015370e+06
...,...
2023-02-21,9.620096e+05
2023-02-22,9.600770e+05
2023-02-23,9.635362e+05


Compare DJIA with the rest of the algorithms

In [110]:
result = pd.DataFrame()
result = pd.merge(result, df_result_a2c, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_result_ddpg, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_result_td3, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_result_ppo, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_result_sac, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_dji, how='outer', left_index=True, right_index=True)
display(result)

,a2c,ddpg,td3,ppo,sac,dji
date,,,,,,
2021-10-01,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06
2021-10-04,9.984603e+05,9.970106e+05,9.969719e+05,9.996348e+05,9.981069e+05,9.905746e+05
2021-10-05,1.002701e+06,1.003180e+06,1.002713e+06,1.000179e+06,1.000426e+06,9.996566e+05
2021-10-06,1.003508e+06,1.005602e+06,1.006578e+06,1.000728e+06,1.002083e+06,1.002637e+06
2021-10-07,1.009787e+06,1.016261e+06,1.015370e+06,1.001783e+06,1.010119e+06,1.012483e+06
...,...,...,...,...,...,...
2023-02-21,9.594672e+05,1.029606e+06,9.620096e+05,8.420724e+05,1.018812e+06,9.651327e+05
2023-02-22,9.567258e+05,1.027130e+06,9.600770e+05,8.389642e+05,1.016171e+06,9.626710e+05
2023-02-23,9.589747e+05,1.031785e+06,9.635362e+05,8.441874e+05,1.021490e+06,9.658412e+05


Save and Plot!

In [111]:
plt.rcParams["figure.figsize"] = (15,5)
plt.figure();
result.plot();
plt.savefig(f'resultadosTFG/set_5-/AlgorithmsVsDJI.png')
plt.close()